In [8]:
from openai import OpenAI
import openai
import os

os.environ["OPENAI_API_KEY"] = ""

In [9]:
Model = "text-embedding-ada-002"

client  = OpenAI()

res = client.embeddings.create(
    input = [
        "Sample document text goes here",
        "another sample tedxt"
    ],
    model=Model
)
res

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.003111546393483877, 0.011806019581854343, -0.005178610794246197, -0.027102630585432053, -0.01637478731572628, 0.03250698372721672, -0.016293924301862717, -0.0010234242072328925, -0.02591663785278797, -0.006482528988271952, 0.020054060965776443, 0.016603900119662285, -0.009097103960812092, 0.02347726747393608, -0.010235927067697048, 0.013490668497979641, 0.02521582506597042, -0.016873443499207497, 0.012136210687458515, -0.01622653752565384, -0.00435313256457448, -0.0064488360658288, -0.0043868254870176315, 0.020835736766457558, -0.010539163835346699, -0.0037702443078160286, 0.013659133575856686, -0.02632095478475094, -0.00043042778270319104, -0.0021361347753554583, 0.005896271206438541, -0.010047245770692825, -0.028167329728603363, -0.016172628849744797, -0.004258792381733656, 0.0074865794740617275, -0.00291275791823864, -0.03150967136025429, 0.02402983233332634, -0.03339647874236107, -0.00038178355316631496, 0.013079614378511906, 0.

In [14]:
len(res.data[0].embedding), len(res.data[1].embedding) 

(1536, 1536)

In [19]:
embeds = [i.embedding for i in res.data]
embeds

[[-0.003111546393483877,
  0.011806019581854343,
  -0.005178610794246197,
  -0.027102630585432053,
  -0.01637478731572628,
  0.03250698372721672,
  -0.016293924301862717,
  -0.0010234242072328925,
  -0.02591663785278797,
  -0.006482528988271952,
  0.020054060965776443,
  0.016603900119662285,
  -0.009097103960812092,
  0.02347726747393608,
  -0.010235927067697048,
  0.013490668497979641,
  0.02521582506597042,
  -0.016873443499207497,
  0.012136210687458515,
  -0.01622653752565384,
  -0.00435313256457448,
  -0.0064488360658288,
  -0.0043868254870176315,
  0.020835736766457558,
  -0.010539163835346699,
  -0.0037702443078160286,
  0.013659133575856686,
  -0.02632095478475094,
  -0.00043042778270319104,
  -0.0021361347753554583,
  0.005896271206438541,
  -0.010047245770692825,
  -0.028167329728603363,
  -0.016172628849744797,
  -0.004258792381733656,
  0.0074865794740617275,
  -0.00291275791823864,
  -0.03150967136025429,
  0.02402983233332634,
  -0.03339647874236107,
  -0.000381783553166

In [18]:
import pinecone

[Embedding(embedding=[-0.003111546393483877, 0.011806019581854343, -0.005178610794246197, -0.027102630585432053, -0.01637478731572628, 0.03250698372721672, -0.016293924301862717, -0.0010234242072328925, -0.02591663785278797, -0.006482528988271952, 0.020054060965776443, 0.016603900119662285, -0.009097103960812092, 0.02347726747393608, -0.010235927067697048, 0.013490668497979641, 0.02521582506597042, -0.016873443499207497, 0.012136210687458515, -0.01622653752565384, -0.00435313256457448, -0.0064488360658288, -0.0043868254870176315, 0.020835736766457558, -0.010539163835346699, -0.0037702443078160286, 0.013659133575856686, -0.02632095478475094, -0.00043042778270319104, -0.0021361347753554583, 0.005896271206438541, -0.010047245770692825, -0.028167329728603363, -0.016172628849744797, -0.004258792381733656, 0.0074865794740617275, -0.00291275791823864, -0.03150967136025429, 0.02402983233332634, -0.03339647874236107, -0.00038178355316631496, 0.013079614378511906, 0.007250729016959667, -0.005586

In [35]:
from pinecone import Pinecone, ServerlessSpec, PodSpec

pinecone = Pinecone(api_key = "7b4a66f0-9542-415b-a5e2-8af79aafc828")

# if "openai" not in pinecone.list_indexes():
#     pinecone.create_index("openai", dimension = len(embeds[0]), 
#                           spec = ServerlessSpec(
#                               cloud = "gcp-starter",
#                               region = "us-central1"
#                           ))
index = pinecone.Index("openai")

In [36]:
from datasets import load_dataset

trec = load_dataset("trec", split = "train[:1000]")
trec

Generating test split: 100%|██████████| 500/500 [00:00<00:00, 250077.75 examples/s]


Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 1000
})

In [37]:
trec[0]

{'text': 'How did serfdom develop in and then leave Russia ?',
 'coarse_label': 2,
 'fine_label': 26}

In [40]:
from tqdm.auto import tqdm
count = 0
batch_size = 32
for i in tqdm(range(0, len(trec), batch_size)):
    i_end = min(i+batch_size, len(trec["text"]))

    lines_batch = trec["text"][i:i_end]
    ids_batch = [str(n) for n in range(i, i_end)]
    res = client.embeddings.create(input = lines_batch, model= Model)
    embeds = [r.embedding for r in res.data]
    meta = [{"text": line} for line in lines_batch]

    to_upsert = zip(ids_batch, embeds, meta)

    index.upsert(vectors = list(to_upsert))

100%|██████████| 32/32 [11:06<00:00, 20.82s/it]


In [47]:
query = "What caused the 1929 Great Depression?"

xq = client.embeddings.create(input = query, model = Model).data[0].embedding
xq

[-0.016057511791586876,
 -0.0330347865819931,
 0.03712261840701103,
 -0.020988458767533302,
 -0.046601276844739914,
 0.00575490016490221,
 -0.003800405655056238,
 0.00931897759437561,
 -0.01391140092164278,
 -0.023338962346315384,
 -0.005713382735848427,
 0.026264315471053123,
 0.0076199728064239025,
 0.012084650807082653,
 0.003102600108832121,
 0.030454343184828758,
 0.0213844683021307,
 0.0076199728064239025,
 0.016031963750720024,
 -0.012455111369490623,
 -0.011011595837771893,
 0.01371978409588337,
 0.0009588838438503444,
 -0.008558896370232105,
 -0.004263480193912983,
 0.005013980437070131,
 0.018024781718850136,
 -0.015137750655412674,
 -0.004311384633183479,
 -0.008175662718713284,
 -0.012193234637379646,
 -0.02549784816801548,
 -0.03709707036614418,
 -0.0033788480795919895,
 -0.019698236137628555,
 -0.0006906199268996716,
 -0.00722396420314908,
 0.0057453191839158535,
 -0.001702198525890708,
 -0.023849939927458763,
 -0.009216781705617905,
 0.005173661280423403,
 -0.01192496996

In [48]:
res = index.query(vector = [xq], top_k = 5, include_metadata= True)
res

{'matches': [{'id': '932',
              'metadata': {'text': 'Why did the world enter a global '
                                   'depression in 1929 ?'},
              'score': 0.913739204,
              'values': []},
             {'id': '787',
              'metadata': {'text': "When was `` the Great Depression '' ?"},
              'score': 0.867940068,
              'values': []},
             {'id': '400',
              'metadata': {'text': 'What crop failure caused the Irish Famine '
                                   '?'},
              'score': 0.810628116,
              'values': []},
             {'id': '775',
              'metadata': {'text': 'What historical event happened in Dogtown '
                                   'in 1899 ?'},
              'score': 0.796658,
              'values': []},
             {'id': '481',
              'metadata': {'text': 'What caused the Lynmouth floods ?'},
              'score': 0.791251898,
              'values': []}],
 'namespace

In [49]:
query = "What caused the recession in 1929"

xq = client.embeddings.create(input = query, model = Model).data[0].embedding
xq
res = index.query(vector = [xq], top_k = 5, include_metadata= True)
res

{'matches': [{'id': '932',
              'metadata': {'text': 'Why did the world enter a global '
                                   'depression in 1929 ?'},
              'score': 0.906015337,
              'values': []},
             {'id': '787',
              'metadata': {'text': "When was `` the Great Depression '' ?"},
              'score': 0.830979705,
              'values': []},
             {'id': '400',
              'metadata': {'text': 'What crop failure caused the Irish Famine '
                                   '?'},
              'score': 0.797391474,
              'values': []},
             {'id': '775',
              'metadata': {'text': 'What historical event happened in Dogtown '
                                   'in 1899 ?'},
              'score': 0.792336464,
              'values': []},
             {'id': '481',
              'metadata': {'text': 'What caused the Lynmouth floods ?'},
              'score': 0.789177358,
              'values': []}],
 'namesp

In [50]:
query = "What was the cause of the major recession in 20th century"

xq = client.embeddings.create(input = query, model = Model).data[0].embedding
xq
res = index.query(vector = [xq], top_k = 5, include_metadata= True)
res

{'matches': [{'id': '932',
              'metadata': {'text': 'Why did the world enter a global '
                                   'depression in 1929 ?'},
              'score': 0.87853235,
              'values': []},
             {'id': '787',
              'metadata': {'text': "When was `` the Great Depression '' ?"},
              'score': 0.826821506,
              'values': []},
             {'id': '400',
              'metadata': {'text': 'What crop failure caused the Irish Famine '
                                   '?'},
              'score': 0.797631145,
              'values': []},
             {'id': '969',
              'metadata': {'text': 'What are some of the significant '
                                   'historical events of the 1990s ?'},
              'score': 0.785506666,
              'values': []},
             {'id': '835',
              'metadata': {'text': 'What were popular songs and types of songs '
                                   'in the 1920s ?'},

In [52]:
query = "Why was there a long-term economic downturn in the early 20th century"

xq = client.embeddings.create(input = query, model = Model).data[0].embedding
xq
res = index.query(vector = [xq], top_k = 5, include_metadata= True)
res

{'matches': [{'id': '932',
              'metadata': {'text': 'Why did the world enter a global '
                                   'depression in 1929 ?'},
              'score': 0.88830328,
              'values': []},
             {'id': '787',
              'metadata': {'text': "When was `` the Great Depression '' ?"},
              'score': 0.834873915,
              'values': []},
             {'id': '400',
              'metadata': {'text': 'What crop failure caused the Irish Famine '
                                   '?'},
              'score': 0.792694807,
              'values': []},
             {'id': '262',
              'metadata': {'text': 'When did World War I start ?'},
              'score': 0.790573478,
              'values': []},
             {'id': '864',
              'metadata': {'text': 'When did the Dow first reach ?'},
              'score': 0.787006259,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}